In [24]:
import numpy as np
import pandas as pd
import yfinance as yf

# -----------------------------
# 1) Datos y matriz de correlación (robusta)
# -----------------------------
# Pon aquí tu lista ~100
tickers = [...]  # <- tu lista

prices = yf.download(tickers, start='2022-01-01', end='2025-09-03')['Close']

# Relleno razonable para huecos cortos; si un ticker está casi vacío seguirá NaN
prices = prices.ffill().bfill()

rets = prices.pct_change()

# Quita filas totalmente vacías (festivos/zonas horarias)
rets = rets.dropna(how='all')

# Filtra columnas con suficiente cobertura (>=95% de observaciones no NaN)
min_cov = 0.95
keep = rets.notna().mean() >= min_cov
rets = rets.loc[:, keep]

# Si quedaron muy pocas series, avisar
if rets.shape[1] < 4:
    raise ValueError(f"Solo {rets.shape[1]} series útiles tras limpieza. Agrega más tickers o reduce min_cov.")

# Correlación con mínimo de pares (80% del total de días válidos)
min_pairs = int(0.8 * len(rets))
corr = rets.corr(min_periods=min_pairs).values
names = rets.columns.to_numpy()
n = corr.shape[0]

# Reemplaza NaN de la correlación por un número alto (penaliza combinaciones inválidas)
corr = np.where(np.isnan(corr), 1.5, corr)
np.fill_diagonal(corr, 1.0)  # por claridad

# ------------------------------------
# 2) Utilidades
# ------------------------------------
def subset_score(cmat, idx4):
    """Suma bajo la diagonal (equivalente a suma de pares i<j)."""
    sub = cmat[np.ix_(idx4, idx4)]
    return np.tril(sub, -1).sum()

def greedy_low_corr(cmat, k=4, restarts=200, seed=42):
    """
    Greedy con múltiples reinicios.
    - Semilla 0: par global menos correlacionado.
    - Resto: semilla aleatoria + mejor compañero.
    Devuelve (idx_list, score) o (None, None) si no hay factible.
    """
    rng = np.random.default_rng(seed)
    n = cmat.shape[0]
    best_idx, best_score = None, None

    # "Infla" la diagonal para evitar autoelección
    diag_inflate = cmat + np.eye(n) * 2.0

    # Precalcula el par global mínimo (ignorando la diagonal inflada)
    flat = diag_inflate.ravel()
    i0 = int(np.argmin(flat) // n)
    j0 = int(np.argmin(flat)  % n)

    for r in range(restarts):
        if r == 0:
            current = [i0, j0]
        else:
            i = int(rng.integers(0, n))
            j = int(np.argmin(diag_inflate[i]))
            current = [i, j]

        # Completa hasta k con el que menos suma de correlaciones aporte al set actual
        while len(current) < k:
            mask = np.ones(n, dtype=bool)
            mask[current] = False
            candidates = np.where(mask)[0]
            # Suma corr del candidato vs el set actual
            scores = cmat[np.ix_(candidates, current)].sum(axis=1)
            pick = candidates[int(np.argmin(scores))]
            current.append(int(pick))

        sc = subset_score(cmat, current)
        if (best_score is None) or (sc < best_score):
            best_idx, best_score = current[:], sc

    return best_idx, best_score

def local_improve(cmat, idx_set, max_iter=200):
    """Mejora 1-opt: intenta reemplazar 1 dentro por 1 fuera si mejora el score."""
    if idx_set is None:
        return None, None
    n = cmat.shape[0]
    current = idx_set[:]
    best = subset_score(cmat, current)

    for _ in range(max_iter):
        improved = False
        outside = [i for i in range(n) if i not in current]
        for pos in range(len(current)):
            base_others = current[:pos] + current[pos+1:]
            # El mejor candidato externo respecto a los otros 3
            cand_scores = cmat[np.ix_(outside, base_others)].sum(axis=1)
            best_cand = outside[int(np.argmin(cand_scores))]
            trial = base_others + [best_cand]
            trial_sc = subset_score(cmat, trial)
            if trial_sc + 1e-12 < best:
                current, best = trial[:], trial_sc
                improved = True
                break
        if not improved:
            break
    return current, best

# ------------------------------------
# 3) Ejecutar búsqueda
# ------------------------------------
seed_set, seed_score = greedy_low_corr(corr, k=4, restarts=500, seed=7)
if seed_set is None:
    raise RuntimeError("No se pudo construir una semilla válida. Revisa NaN en la correlación o cobertura de datos.")

final_idx, final_score = local_improve(corr, seed_set, max_iter=300)

print("Selección inicial (greedy):", names[seed_set], "score:", round(seed_score, 6))
print("Selección final (mejorada):", names[final_idx], "score:", round(final_score, 6))

sub_corr = pd.DataFrame(
    corr[np.ix_(final_idx, final_idx)],
    index=names[final_idx],
    columns=names[final_idx]
)
print("\nCorrelaciones del set encontrado:")
print(sub_corr.round(3))


C:\Users\samue\AppData\Local\Temp\ipykernel_17340\3164754344.py:11: FutureWarning: YF.download() has changed argument auto_adjust default to True
  prices = yf.download(tickers, start='2022-01-01', end='2025-09-03')['Close']


TypeError: expected string or bytes-like object, got 'ellipsis'